In [6]:
import numpy as np


Logistic Regression algorithm

In [7]:
class LogisticRegressionMultiClass:
    def __init__(self, lr=0.2, n_epochs=40, mini_batch_size=256):
        self.lr = lr
        self.n_epochs = n_epochs
        self.mini_batch_size = mini_batch_size
        self.mean = None
        self.std = None
        self.weights = None
        self.bias = None
        self.classes = None

    def sigmoid(self, z):
        z = np.clip(z, -50, 50)
        return 1 / (1 + np.exp(-z))

    def standardize(self, X, train=True):
        if train:
            self.mean = X.mean(axis=0)
            self.std = X.std(axis=0) + 1e-8
        return (X - self.mean) / self.std

    def fit(self, X, y):
        # STANDARDIZE ONCE
        X = self.standardize(X, train=True)
        n_samples, n_features = X.shape
        
        # Unique classes 0–9
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        # Parameters for each classifier (one-vs-rest)
        self.weights = np.zeros((n_classes, n_features))
        self.bias = np.zeros(n_classes)

        # Train one classifier per digit
        for idx, c in enumerate(self.classes):
            print(f"Training class {c} vs rest")

            y_binary = (y == c).astype(int)

            w = np.zeros(n_features)
            b = 0

            for epoch in range(self.n_epochs):

                # Shuffle indices
                indices = np.arange(n_samples)
                np.random.shuffle(indices)
                X_shuffled = X[indices]
                y_shuffled = y_binary[indices]

                # Mini-batch SGD
                for i in range(0, n_samples, self.mini_batch_size):
                    X_batch = X_shuffled[i:i+self.mini_batch_size]
                    y_batch = y_shuffled[i:i+self.mini_batch_size]

                    z = np.dot(X_batch, w) + b
                    y_pred = self.sigmoid(z)

                    dw = np.dot(X_batch.T, (y_pred - y_batch)) / len(y_batch)
                    db = np.sum(y_pred - y_batch) / len(y_batch)

                    w -= self.lr * dw
                    b -= self.lr * db

            self.weights[idx] = w
            self.bias[idx] = b

    def predict(self, X):
        X = self.standardize(X, train=False)
        logits = np.dot(X, self.weights.T) + self.bias
        probs = self.sigmoid(logits)
        return np.argmax(probs, axis=1)

SVM algorithm

In [8]:
class MulticlassSVM:
    def __init__(self, learning_rate=0.001, lambda_p=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_p = lambda_p
        self.n_iters = n_iters
        self.W = None     # shape: (K, D)
        self.b = None     # shape: (K,)

    def _binary_svm_update(self, X, y, w, b):
        """
        Performs SGD updates for ONE binary classifier.
        X: (N, D)
        y: (N,) labels in {-1, +1}
        """
        n_samples, n_features = X.shape

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, w) - b) >= 1

                if condition:
                    # only regularization affects gradient
                    w -= self.lr * (self.lambda_p * w)
                else:
                    # hinge loss + regularization
                    w -= self.lr * (self.lambda_p * w - y[idx] * x_i)
                    b -= self.lr * y[idx]

        return w, b

    def fit(self, X, y):
        """
        Train K binary classifiers (one-vs-rest) using the above SGD procedure.
        """
        N, D = X.shape
        classes = np.unique(y)
        K = len(classes)

        # initialize parameters
        self.W = np.zeros((K, D))
        self.b = np.zeros(K)

        # Train each class separately
        for k in classes:
            print(f"Training class {k} vs rest...")

            # Convert labels to +1 (class k) and -1 (all others)
            y_binary = np.where(y == k, 1, -1)

            # Extract w_k, b_k
            w_k = self.W[k].copy()
            b_k = self.b[k].copy()

            # Train binary classifier
            w_k, b_k = self._binary_svm_update(X, y_binary, w_k, b_k)

            # Store results
            self.W[k] = w_k
            self.b[k] = b_k

    def predict(self, X):
        # Compute scores for each class
        scores = X.dot(self.W.T) - self.b
        return np.argmax(scores, axis=1)


XGBoost algorithm

In [9]:

# Decision Stump for XGBoost
class DecisionStumpXGB:

    def __init__(self, depth=1, max_depth=1, reg_lambda=1.0, gamma=0.0, subsample_features=1.0, n_thresholds=10):
        self.feature_index = None
        self.threshold = None
        self.value = 0.0
        self.left_stump = None
        self.right_stump = None
        self.is_leaf = False
        self.is_valid = False

        self.depth = depth
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.subsample_features = subsample_features
        self.n_thresholds = n_thresholds

    def leaf_value(self, grad, hess):
        return -np.sum(grad) / (np.sum(hess) + self.reg_lambda)

    def fit(self, X, grad, hess):
        m, n = X.shape
        assert grad.shape[0] == m

        if self.depth >= self.max_depth:
            self.is_leaf = True
            self.is_valid = True
            self.value = self.leaf_value(grad, hess)
            return

        best_gain = -float('inf')
        best_feature = None
        best_threshold = None
        best_masks = None
        n_sub = max(1, int(self.subsample_features * n))
        feature_indices = np.random.choice(n, n_sub, replace=False)

        for feature_index in feature_indices:
            col = X[:, feature_index]
            if np.isnan(col).any():
                continue

            quantiles = np.linspace(0, 1, num=self.n_thresholds)
            thresholds = np.unique(np.quantile(col, quantiles))

            for threshold in thresholds:
                left_mask = col <= threshold
                right_mask = ~left_mask

                if left_mask.sum() == 0 or right_mask.sum() == 0:
                    continue

                G_left, H_left = np.sum(grad[left_mask]), np.sum(hess[left_mask])
                G_right, H_right = np.sum(grad[right_mask]), np.sum(hess[right_mask])

                gain = 0.5 * (
                    (G_left**2 / (H_left + self.reg_lambda)) + 
                    (G_right**2 / (H_right + self.reg_lambda)) - 
                    ((G_left + G_right)**2 / (H_left + H_right + self.reg_lambda))
                ) - self.gamma

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = threshold
                    best_masks = (left_mask, right_mask)

        if best_feature is None or best_gain <= 0:
            self.is_leaf = True
            self.is_valid = True
            self.value = self.leaf_value(grad, hess)
            return

        self.feature_index = best_feature
        self.threshold = best_threshold
        self.is_valid = True
        left_mask, right_mask = best_masks

        self.left_stump = DecisionStumpXGB(
            depth=self.depth + 1, max_depth=self.max_depth,
            reg_lambda=self.reg_lambda, gamma=self.gamma,
            subsample_features=self.subsample_features, n_thresholds=self.n_thresholds
        )

        self.right_stump = DecisionStumpXGB(
            depth=self.depth + 1, max_depth=self.max_depth,
            reg_lambda=self.reg_lambda, gamma=self.gamma,
            subsample_features=self.subsample_features, n_thresholds=self.n_thresholds
        )

        self.left_stump.fit(X[left_mask], grad[left_mask], hess[left_mask])
        self.right_stump.fit(X[right_mask], grad[right_mask], hess[right_mask])

    def predict(self, X):
        if not self.is_valid:
            return np.zeros(X.shape[0], dtype=float)

        if self.is_leaf or self.feature_index is None:
            return np.full(X.shape[0], self.value, dtype=float)

        feature = X[:, self.feature_index]
        left_mask = feature <= self.threshold
        right_mask = ~left_mask
        preds = np.zeros(X.shape[0], dtype=float)

        if self.left_stump:
            preds[left_mask] = self.left_stump.predict(X[left_mask])
        else:
            preds[left_mask] = self.value

        if self.right_stump:
            preds[right_mask] = self.right_stump.predict(X[right_mask])
        else:
            preds[right_mask] = self.value

        return preds

# Binary XGBoost Classifier
class XGBoostClassifier:

    def __init__(self, n_estimators=40, learning_rate=0.1, max_depth=3, reg_lambda=1.0, gamma=0.1,
                 subsample_features=0.3, n_thresholds=10, proba_threshold=0.5):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.reg_lambda = reg_lambda
        self.gamma = gamma
        self.subsample_features = subsample_features
        self.n_thresholds = n_thresholds
        self.proba_threshold = proba_threshold
        self.trees = []
        self.init_pred = 0.0

    def sigmoid(self, x):
        x = np.clip(x, -20, 20)  # numerical stability
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y).astype(float)
        m = X.shape[0]

        eps = 1e-15
        y_mean = np.clip(np.mean(y), eps, 1 - eps)
        self.init_pred = np.log(y_mean / (1 - y_mean))
        pred = np.full(m, self.init_pred, dtype=float)

        for _ in range(self.n_estimators):
            prob = self.sigmoid(pred)
            grad = prob - y
            hess = prob * (1 - prob)

            stump = DecisionStumpXGB(
                depth=1,
                max_depth=self.max_depth,
                reg_lambda=self.reg_lambda,
                gamma=self.gamma,
                subsample_features=self.subsample_features,
                n_thresholds=self.n_thresholds
            )
            stump.fit(X, grad, hess)
            update = stump.predict(X)
            pred += self.learning_rate * update
            self.trees.append(stump)

    def predict_proba(self, X):
        X = np.asarray(X)
        m = X.shape[0]
        pred = np.full(m, self.init_pred, dtype=float)
        for stump in self.trees:
            pred += self.learning_rate * stump.predict(X)
        return self.sigmoid(pred)

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= self.proba_threshold).astype(int)

# Multiclass XGBoost Classifier

class XGBoostMulticlass:

    def __init__(self, num_classes=10, **kwargs):
        self.num_classes = num_classes
        self.models = [XGBoostClassifier(**kwargs) for _ in range(num_classes)]

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        for k in range(self.num_classes):
            print(f"Training class {k} vs rest...")
            y_binary = (y == k).astype(int)
            self.models[k].fit(X, y_binary)

    def predict_proba(self, X):
        X = np.asarray(X)
        # Get raw logits from each binary classifier
        logits = np.column_stack([self._model_logits(model, X) for model in self.models])
        # Softmax for calibrated probabilities
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # stability trick
        probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
        return probs

    def _model_logits(self, model, X):
        m = X.shape[0]
        pred = np.full(m, model.init_pred, dtype=float)
        for tree in model.trees:
            pred += model.learning_rate * tree.predict(X)
        return pred

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

KNN algorithm

In [10]:


class KNN:
    def __init__(self, k=5, batch_size=500):
        self.k = k
        self.batch_size = batch_size

    def _normalize(self, X):
        # Scale to 0–1
        X = X.astype(np.float32) / 255.0
        # L2 normalize each sample (very important for cosine distance)
        norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-6
        return X / norms

    def fit(self, X, y):
        self.X_train = self._normalize(X)
        self.y_train = y

    def predict(self, X):
        X = self._normalize(X)
        n_test = X.shape[0]
        predictions = np.zeros(n_test, dtype=int)

        for i in range(0, n_test, self.batch_size):
            X_batch = X[i:i+self.batch_size]

            # Cosine similarity = dot(x, y)
            sim = np.dot(X_batch, self.X_train.T)

            # Convert similarity to distance
            dists = 1 - sim   # smaller = closer

            # Find top-k nearest neighbors
            k_idx = np.argpartition(dists, self.k, axis=1)[:, :self.k]

            k_labels = self.y_train[k_idx]
            k_dists = dists[np.arange(dists.shape[0])[:, None], k_idx]

            # Weight = 1 / distance (closer neighbor gets more power)
            weights = 1 / (k_dists + 1e-6)

            # Weighted vote
            batch_pred = [
                np.bincount(k_labels[row], weights=weights[row]).argmax()
                for row in range(k_labels.shape[0])
            ]

            predictions[i:i+self.batch_size] = batch_pred

        return predictions